In [ ]:
import pandas as pd

from data_tools.static_mappings import specific_to_grouped_attained_education_map

In the [current_education](current_education.ipynb) notebook, we prepared a joint data set for current education, attempting to solve two issues. First, combining separate data sets for primary and higher education, and secondly, estimating the number of agents _not_ currently enrolled in education.

In the [education_attainment](education_attainment.ipynb) notebook, we subsequently attempted to join the education attainment data over this data. However, for the education attainment data, we have neighborhood margins available, which cannot be fitted properly if conditioned on the current education for which we have no neighborhood margins.

This means we applied the wrong order, and we now need to condition the current education on education attainment again. This notebook is meant to investigate if we can use the fractions already calculated in the education attainment data, or if we need to perform the reverse approach from the [education_attainment](education_attainment.ipynb) notebook manually.

In [ ]:
df_current_edu = pd.read_pickle('../processed/prepared_current_education.pkl')
df_current_edu

,age,gender,migration_background,current_education,count
0,10,female,Dutch,Basisonderwijs,1166.152771
1,10,female,NonWestern,Basisonderwijs,1261.690120
2,10,female,Western,Basisonderwijs,463.157108
3,11,female,Dutch,Basisonderwijs,1095.562410
4,11,female,NonWestern,Basisonderwijs,1185.316627
...,...,...,...,...,...
4423,9,female,NonWestern,not_enrolled,0.000000
4424,9,female,Western,not_enrolled,0.000000
4425,9,male,Dutch,not_enrolled,0.000000
4426,9,male,NonWestern,not_enrolled,0.000000


In [ ]:
df_current_edu.age.unique()

array(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '30-35',
       '35-40', '40-45', '45-50', '50-55', '55-60', '60-65', '65-70',
       '70-75', '75-80', '80-85', '85-90', '90-95', '95+', '0-4', '4',
       '5', '6', '7', '8', '9'], dtype=object)

In [ ]:
df_absolved_edu = pd.read_pickle('../processed/prepared_absolved_education.pkl')
df_absolved_edu

,age,gender,absolved_education,current_education,count,absolved_edu_3_cats
0,15-25,female,111 Basisonderwijs,Assistentopleiding (niveau 1),3978.504474,low
1,15-25,female,111 Basisonderwijs,Basisberoepsopleiding (niveau 2),23122.383529,low
2,15-25,female,111 Basisonderwijs,Praktijkonderwijs,6716.400026,low
3,15-25,female,111 Basisonderwijs,Vavo,4855.486643,low
4,15-25,female,111 Basisonderwijs,Vmbo basis-kaderberoeps 3-4,21111.741483,low
...,...,...,...,...,...,...
6151,9,male,no_education,Hoger beroepsonderwijs,0.000000,low
6152,9,male,no_education,Wetenschappelijk onderwijs,0.000000,low
6153,9,male,no_education,Basisonderwijs,2886.000000,low
6154,9,male,no_education,Speciaal basisonderwijs,147.000000,low


In [ ]:
df_absolved_edu.age.unique()

array(['15-25', '25-35', '35-45', '45-55', '55-65', '65-75', '75+', '0-4',
       '10', '11', '12', '13', '14', '4', '5', '6', '7', '8', '9'],
      dtype=object)

We apply the same age trick again to map the ages. The current education ages encompass all of the absolved education ages, so we map from the first to the second

In [ ]:
age_map = {
              str(i): str(i) for i in range(4, 15)
          } | {
              str(i): f'{(i - 5) // 10 * 10 + 5}-{(i - 5) // 10 * 10 + 15}' for i in range(15, 30)
          } | {
              f'{i}-{i + 5}': f'{i}-{i + 10}' for i in range(15, 75, 10)
          } | {
              f'{i}-{i + 5}': f'{i - 5}-{i + 5}' for i in range(20, 80, 10)
          } | {
              '0-4': '0-4', '75-80': '75+', '80-85': '75+', '85-90': '75+', '90-95': '75+', '95+': '75+'
          }
age_map

{'4': '4',
 '5': '5',
 '6': '6',
 '7': '7',
 '8': '8',
 '9': '9',
 '10': '10',
 '11': '11',
 '12': '12',
 '13': '13',
 '14': '14',
 '15': '15-25',
 '16': '15-25',
 '17': '15-25',
 '18': '15-25',
 '19': '15-25',
 '20': '15-25',
 '21': '15-25',
 '22': '15-25',
 '23': '15-25',
 '24': '15-25',
 '25': '25-35',
 '26': '25-35',
 '27': '25-35',
 '28': '25-35',
 '29': '25-35',
 '15-20': '15-25',
 '25-30': '25-35',
 '35-40': '35-45',
 '45-50': '45-55',
 '55-60': '55-65',
 '65-70': '65-75',
 '20-25': '15-25',
 '30-35': '25-35',
 '40-45': '35-45',
 '50-55': '45-55',
 '60-65': '55-65',
 '70-75': '65-75',
 '0-4': '0-4',
 '75-80': '75+',
 '80-85': '75+',
 '85-90': '75+',
 '90-95': '75+',
 '95+': '75+'}

In [ ]:
tst = pd.DataFrame(df_current_edu.age.unique())
tst.loc[:, "age"] = tst[0].map(age_map)
tst

,0,age
0,10,10
1,11,11
2,12,12
3,13,13
4,14,14
5,15,15-25
6,16,15-25
7,17,15-25
8,18,15-25
9,19,15-25


In [ ]:
df_current_edu.loc[:, "absolved_edu_age"] = df_current_edu.age.map(age_map)
df_merged = df_current_edu.merge(df_absolved_edu, how='left',
                                 left_on=['absolved_edu_age', 'gender', 'current_education'],
                                 right_on=['age', 'gender', 'current_education']).drop(['age_y', 'absolved_edu_3_cats'],
                                                                                       axis=1)
df_merged

,age_x,gender,migration_background,current_education,count_x,absolved_edu_age,absolved_education,count_y
0,10,female,Dutch,Basisonderwijs,1166.152771,10,111 Basisonderwijs,0.0
1,10,female,Dutch,Basisonderwijs,1166.152771,10,"121 Vmbo-b/k, mbo1",0.0
2,10,female,Dutch,Basisonderwijs,1166.152771,10,"122 Vmbo-g/t, havo-, vwo-onderbouw",0.0
3,10,female,Dutch,Basisonderwijs,1166.152771,10,211 Mbo2 en mbo3,0.0
4,10,female,Dutch,Basisonderwijs,1166.152771,10,212 Mbo4,0.0
...,...,...,...,...,...,...,...,...
39847,9,male,Western,not_enrolled,0.000000,9,212 Mbo4,0.0
39848,9,male,Western,not_enrolled,0.000000,9,"213 Havo, vwo",0.0
39849,9,male,Western,not_enrolled,0.000000,9,"311 Hbo-, wo-bachelor",0.0
39850,9,male,Western,not_enrolled,0.000000,9,"321 Hbo-, wo-master, doctor",0.0


If we group by all values present in the original data frame, we can calculate the fraction of each level of absolved education in each group. That should give us the current education sizes.

However, some groups have no values in the absolved education count, resulting in a `NaN` fraction. In case there is a value for current education count, we fill those `NaN`-values with 1 over the total number of absolved levels of education there are: 

In [ ]:
fill_na_with = 1 / len(df_merged.absolved_education.unique())
fill_na_with

0.1111111111111111

In [ ]:
df_merged.loc[:, 'frac_y'] = df_merged.groupby(
        ['gender', 'age_x', 'migration_background', 'current_education']).count_y.transform(
        lambda x: x / x.sum()).fillna(fill_na_with)
df_merged

,age_x,gender,migration_background,current_education,count_x,absolved_edu_age,absolved_education,count_y,frac_y
0,10,female,Dutch,Basisonderwijs,1166.152771,10,111 Basisonderwijs,0.0,0.000000
1,10,female,Dutch,Basisonderwijs,1166.152771,10,"121 Vmbo-b/k, mbo1",0.0,0.000000
2,10,female,Dutch,Basisonderwijs,1166.152771,10,"122 Vmbo-g/t, havo-, vwo-onderbouw",0.0,0.000000
3,10,female,Dutch,Basisonderwijs,1166.152771,10,211 Mbo2 en mbo3,0.0,0.000000
4,10,female,Dutch,Basisonderwijs,1166.152771,10,212 Mbo4,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
39847,9,male,Western,not_enrolled,0.000000,9,212 Mbo4,0.0,0.111111
39848,9,male,Western,not_enrolled,0.000000,9,"213 Havo, vwo",0.0,0.111111
39849,9,male,Western,not_enrolled,0.000000,9,"311 Hbo-, wo-bachelor",0.0,0.111111
39850,9,male,Western,not_enrolled,0.000000,9,"321 Hbo-, wo-master, doctor",0.0,0.111111


In [ ]:
df_merged.loc[:, 'count_z'] = df_merged.count_x * df_merged.frac_y
df_merged

,age_x,gender,migration_background,current_education,count_x,absolved_edu_age,absolved_education,count_y,frac_y,count_z
0,10,female,Dutch,Basisonderwijs,1166.152771,10,111 Basisonderwijs,0.0,0.000000,0.0
1,10,female,Dutch,Basisonderwijs,1166.152771,10,"121 Vmbo-b/k, mbo1",0.0,0.000000,0.0
2,10,female,Dutch,Basisonderwijs,1166.152771,10,"122 Vmbo-g/t, havo-, vwo-onderbouw",0.0,0.000000,0.0
3,10,female,Dutch,Basisonderwijs,1166.152771,10,211 Mbo2 en mbo3,0.0,0.000000,0.0
4,10,female,Dutch,Basisonderwijs,1166.152771,10,212 Mbo4,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
39847,9,male,Western,not_enrolled,0.000000,9,212 Mbo4,0.0,0.111111,0.0
39848,9,male,Western,not_enrolled,0.000000,9,"213 Havo, vwo",0.0,0.111111,0.0
39849,9,male,Western,not_enrolled,0.000000,9,"311 Hbo-, wo-bachelor",0.0,0.111111,0.0
39850,9,male,Western,not_enrolled,0.000000,9,"321 Hbo-, wo-master, doctor",0.0,0.111111,0.0


In [ ]:
df_current_edu['count'].sum(), df_merged.count_z.sum()

(533885.0, 533885.0)

In [ ]:
df_current_edu['count'].sum() - df_merged.count_z.sum()

0.0

In [ ]:
df_merged.loc[:, 'sum_z'] = df_merged.groupby(
        ['gender', 'age_x', 'migration_background', 'current_education']).count_z.transform('sum')
df_merged

,age_x,gender,migration_background,current_education,count_x,absolved_edu_age,absolved_education,count_y,frac_y,count_z,sum_z
0,10,female,Dutch,Basisonderwijs,1166.152771,10,111 Basisonderwijs,0.0,0.000000,0.0,1166.152771
1,10,female,Dutch,Basisonderwijs,1166.152771,10,"121 Vmbo-b/k, mbo1",0.0,0.000000,0.0,1166.152771
2,10,female,Dutch,Basisonderwijs,1166.152771,10,"122 Vmbo-g/t, havo-, vwo-onderbouw",0.0,0.000000,0.0,1166.152771
3,10,female,Dutch,Basisonderwijs,1166.152771,10,211 Mbo2 en mbo3,0.0,0.000000,0.0,1166.152771
4,10,female,Dutch,Basisonderwijs,1166.152771,10,212 Mbo4,0.0,0.000000,0.0,1166.152771
...,...,...,...,...,...,...,...,...,...,...,...
39847,9,male,Western,not_enrolled,0.000000,9,212 Mbo4,0.0,0.111111,0.0,0.000000
39848,9,male,Western,not_enrolled,0.000000,9,"213 Havo, vwo",0.0,0.111111,0.0,0.000000
39849,9,male,Western,not_enrolled,0.000000,9,"311 Hbo-, wo-bachelor",0.0,0.111111,0.0,0.000000
39850,9,male,Western,not_enrolled,0.000000,9,"321 Hbo-, wo-master, doctor",0.0,0.111111,0.0,0.000000


In [ ]:
df_merged[abs(df_merged.count_x - df_merged.sum_z) > 0.001]

,age_x,gender,migration_background,current_education,count_x,absolved_edu_age,absolved_education,count_y,frac_y,count_z,sum_z


In [ ]:
df_merged[df_merged.absolved_education.isna()]

,age_x,gender,migration_background,current_education,count_x,absolved_edu_age,absolved_education,count_y,frac_y,count_z,sum_z


In [ ]:
df_edu = df_merged[
    ["age_x", "gender", "migration_background", "absolved_education", "current_education", "count_z"]].rename(columns={
    'age_x': 'age',
    'count_z': 'count'
})
df_edu

,age,gender,migration_background,absolved_education,current_education,count
0,10,female,Dutch,111 Basisonderwijs,Basisonderwijs,0.0
1,10,female,Dutch,"121 Vmbo-b/k, mbo1",Basisonderwijs,0.0
2,10,female,Dutch,"122 Vmbo-g/t, havo-, vwo-onderbouw",Basisonderwijs,0.0
3,10,female,Dutch,211 Mbo2 en mbo3,Basisonderwijs,0.0
4,10,female,Dutch,212 Mbo4,Basisonderwijs,0.0
...,...,...,...,...,...,...
39847,9,male,Western,212 Mbo4,not_enrolled,0.0
39848,9,male,Western,"213 Havo, vwo",not_enrolled,0.0
39849,9,male,Western,"311 Hbo-, wo-bachelor",not_enrolled,0.0
39850,9,male,Western,"321 Hbo-, wo-master, doctor",not_enrolled,0.0


In [ ]:
df_edu["count"].sum()

533885.0

In [ ]:
df_edu.absolved_education.unique()

array(['111 Basisonderwijs', '121 Vmbo-b/k, mbo1',
       '122 Vmbo-g/t, havo-, vwo-onderbouw', '211 Mbo2 en mbo3',
       '212 Mbo4', '213 Havo, vwo', '311 Hbo-, wo-bachelor',
       '321 Hbo-, wo-master, doctor', 'no_education'], dtype=object)

In [ ]:
df_edu.current_education.unique()

array(['Basisonderwijs', 'Speciaal basisonderwijs', 'Speciale scholen',
       'Vo algemene leerjaren 1-3', 'Vwo 3-6', 'Havo 3-5',
       'Vmbo theoretische-gemengde leerweg 3-4',
       'Vmbo basis-kaderberoeps 3-4', 'Praktijkonderwijs', 'Vavo',
       'Assistentopleiding (niveau 1)',
       'Basisberoepsopleiding (niveau 2)', 'Vakopleiding (niveau 3)',
       'Middenkaderopleiding (niveau 4a)',
       'Specialistenopleiding (niveau 4b)', 'Hoger beroepsonderwijs',
       'Wetenschappelijk onderwijs', 'not_enrolled'], dtype=object)

In [ ]:
df_edu.loc[:, "absolved_edu_3_cats"] = df_edu.absolved_education.replace(
        specific_to_grouped_attained_education_map)

In [ ]:
df_edu.to_pickle('../processed/prepared_education_conditioned_on_absolved_education.pkl')